In [4]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [5]:
path = '/scratch1/nsuresh/BET_BSE_DATA/'

In [6]:
data = pd.read_csv(path + "Label_file.csv")
data.head(5)

,Filename,Recognizable-Facial-Feature,Brain-Feature-Loss
0,IXI369-Guys-0924-T1_bet_03.nii,Yes,No
1,IXI448-HH-2393-T1_bet_07.nii,Yes,No
2,IXI252-HH-1693-T1_bet_08.nii,Yes,No
3,IXI188-Guys-0798-T1_bet_17.nii,Yes,No
4,IXI182-Guys-0792-T1_bet_17.nii,Yes,No


In [7]:
df = data.iloc[: , :-1]
df['Recognizable-Facial-Feature'].value_counts()

Yes    692
No     619
Name: Recognizable-Facial-Feature, dtype: int64

In [32]:
from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath).get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    return volume.astype("float32")


def resize_volume(img):
    depth_factor = 1 / (img.shape[-1] / 64)
    width_factor = 1 / (img.shape[0] / 128)
    height_factor = 1 / (img.shape[1] / 128)
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [8]:
#np_data = np.array([process_scan(path + 'files/' + filepath) for filepath in df['Filename']])

In [29]:
print("Train Size: ", round(1311 * 0.7))
print("Test Size: ", round(1311 * 0.3))

Train Size:  918
Test Size:  393


In [19]:
X_train = np_data[:918, :]
y_train = np_data[918:, :]

In [35]:
data['Recognizable-Facial-Feature'] = data['Recognizable-Facial-Feature'].replace({'Yes': 1, 'No': 0})

In [38]:
X_test = data['Recognizable-Facial-Feature'][:918]
y_test = data['Recognizable-Facial-Feature'][918:]

In [40]:
'''
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)
'''

In [54]:
data.iloc[:, :-1].to_csv('Rec_face.csv', index = False)